# 4.2 - Procesos paralelos


![parallel](images/parallel.png)

$$$$

### Multiprocessing

Veamos en primer lugar [multiprocessing](https://docs.python.org/es/3.9/library/multiprocessing.html). Es una librería de Python que nos permite manejar hilos y procesos. La diferencia entre hilo y proceso es que un hilo ocurre dentro del espacio de memoria de un programa y un proceso es una copia completa del programa, por esta razón, los hilos son rápidos de crear y destruir además de que consumen poca memoria y los procesos son lentos de crear y destruir además de que requieren clonar el espacio de memoria del programa en otro lugar de la RAM, y esto es lento. Ejemplos de esto serían, subrutinas que recogen mensajes de un puerto de comunicaciones y los usan para actuar sobre emails almacenados en un servidor, desde el punto de vista del servidor, el cliente de correo sólo necesita usar el servidor durante un corto plazo de tiempo, porque envía un mensaje al servidor donde le indica lo que el usuario desea hacer, saber si hay mensajes nuevos, borrar un correo, moverlo... El servidor abre un hilo para atender a ese usuario y el hilo sólo vive mientras dure la conexión del usuario, una vez el usuario ha terminado el cliente de correo desconecta hasta nueva acción. Este proceso que he descrito es rápido, ocurre en milisegundos y generalmente se resuelve con hilos porque es más ligero para el sistema operativo y su vida media es especialmente corta, además de que el sistema podrá aceptar ciento o miles de conexiones por segundo y será ligero, rápido y eficiente en esta tarea.

La tendencia actual entre los desarrolladores es hacer una aplicaciones que sean rápidas en un sólo hilo y luego escalar a tantas instancias como sea necesario para cubrir nuestros objetivos de aprovechamiento, estos servidores pueden atender en un sólo proceso a miles o decena de miles de conexiones.

Si queremos realizar un programa que aproveche las diferentes CPUs y pueda realizar múltiples tareas a la vez tenemos muchos mecanismos para llevar esta tarea a cabo. Dependiendo del uso que se quiera dar probablemente queramos usar hilos o procesos, es aquí donde querremos escribir nuestro código con hilos o procesos.

**Hola Mundo**

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
def cuadrado(x):
    return x**2

In [3]:
data = [i for i in range(10000000)]

In [4]:
%%time

seq=[cuadrado(x) for x in data]

seq[:5]

CPU times: user 2.39 s, sys: 46.6 ms, total: 2.43 s
Wall time: 2.43 s


[0, 1, 4, 9, 16]

In [5]:
%%time

map(cuadrado, data)

CPU times: user 1e+03 ns, sys: 0 ns, total: 1e+03 ns
Wall time: 3.1 µs


In [6]:
%%time

seq=list(map(cuadrado, data))
seq[:5]

CPU times: user 2.51 s, sys: 48.3 ms, total: 2.55 s
Wall time: 2.56 s


[0, 1, 4, 9, 16]

In [7]:
import multiprocessing as mp

In [8]:
# movida del Mac M1

mp.get_start_method()

'spawn'

In [9]:
# movida del mac M1, para otros no hace falta

from multiprocessing import get_context

In [10]:
mp.cpu_count()   # nº de nucleos

8

In [11]:
%%time

#pool = mp.Pool(mp.cpu_count())                 # usar todos los nucleos
pool = get_context('fork').Pool(mp.cpu_count()) # para el M1

res=pool.map(cuadrado, data)
pool.close()

res[:5]

CPU times: user 936 ms, sys: 336 ms, total: 1.27 s
Wall time: 1.35 s


[0, 1, 4, 9, 16]

**multiprocessing asíncrono**

`map` consume su iterable convirtiendo el iterable en una lista, dividiéndolo en fragmentos y enviando esos fragmentos a los procesos de trabajo en el Pool. Dividir el iterable en fragmentos funciona mejor que pasar cada elemento en el iterable entre procesos un elemento a la vez, especialmente si el iterable es grande. Sin embargo, convertir el iterable en una lista para dividirlo puede tener un costo de memoria muy alto, ya que la lista completa deberá mantenerse en la memoria.

`imap`/`map_async` no convierte el iterable que le da en una lista, ni lo divide en trozos. Itera sobre el elemento de uno en uno y los envia a un proceso de trabajo distinto. Esto significa que no se toma el golpe de memoria de convertir todo el iterable en una lista, pero también que el rendimiento es más lento para los iterables grandes, debido a la falta de fragmentación. Esto se puede mitigar aumentando el valor predeterminado de 1 en el `chunksize`. Otra gran diferencia de `imap` es que puede comenzar a recibir resultados de los trabajadores tan pronto como estén listos, en lugar de tener que esperar a que todos terminen. 




In [12]:
%%time

pool=mp.Pool(mp.cpu_count())
pool=get_context('fork').Pool(6)  # grupo con 6 cores

res=pool.map_async(cuadrado, data).get()
pool.close()

res[:5]

CPU times: user 1.06 s, sys: 498 ms, total: 1.56 s
Wall time: 1.73 s


[0, 1, 4, 9, 16]

```python
%%time
pool=mp.Pool(mp.cpu_count())   

for x in pool.imap(cuadrado, datos):
    print(x)
    
pool.close()
```

$$$$

$$$$

## Joblib

![joblib](images/joblib.svg)

$$$$

$$$$


[Joblib](https://joblib.readthedocs.io/en/latest/) es una librería de Python que también nos permite paralelizar un programa. En este caso a través de procesos, lo cuál implica, como vimos antes, cierto tiempo para construir el Pool. Lo usaremos principalmente para realizar un bucle sobre una función.

Veamos el Hola Mundo.

**Hola Mundo**

In [13]:
%pip install joblib

Note: you may need to restart the kernel to use updated packages.


In [14]:
from joblib import Parallel, delayed

In [15]:
%%time

paralelo = Parallel(n_jobs=-1, verbose=True)

# -1 en n_jobs es usa todos los nucleos

lst=paralelo(delayed(cuadrado)(e) for e in data)

lst[:5]

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    0.9s
[Parallel(n_jobs=-1)]: Done 16392 tasks      | elapsed:    1.2s
[Parallel(n_jobs=-1)]: Done 1785864 tasks      | elapsed:    8.0s
[Parallel(n_jobs=-1)]: Done 4653064 tasks      | elapsed:   20.8s
[Parallel(n_jobs=-1)]: Done 8339464 tasks      | elapsed:   34.9s


CPU times: user 38.9 s, sys: 1.27 s, total: 40.1 s
Wall time: 41.1 s


[Parallel(n_jobs=-1)]: Done 10000000 out of 10000000 | elapsed:   41.1s finished


[0, 1, 4, 9, 16]

### Ejemplo ESPN

Volvamos de nuevo al ejemplo de scrapeo de la págine de ESPN. Usaremos joblib para realizar una extracción en paralelo de la información.

In [16]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select

import time

import pandas as pd

#from webdriver_manager.chrome import ChromeDriverManager
#PATH=ChromeDriverManager().install()

PATH='driver/chromedriver'

In [17]:
url='https://www.espn.com/soccer/competitions'


driver=webdriver.Chrome(PATH)
driver.get(url)
    
time.sleep(2)

aceptar=driver.find_element(By.XPATH, '//*[@id="onetrust-accept-btn-handler"]')
aceptar.click()

time.sleep(4)

equipos=driver.find_element(By.CSS_SELECTOR, '#fittPageContainer > div.page-container.cf > div > div.layout__column.layout__column--1 > div > div:nth-child(3) > div:nth-child(1) > div > div:nth-child(5) > div > section > div > div > span:nth-child(2) > a')
equipos.click()


time.sleep(2)

equipos_stats_urls=driver.find_elements(By.CSS_SELECTOR, 'a.AnchorLink')

equipos_stats_urls=[e.get_attribute('href') for e in equipos_stats_urls 
                    if 'team/stats' in e.get_attribute('href')]


equipos_stats_urls[:20]

['https://www.espn.com/soccer/team/stats/_/id/3802/afc-wimbledon',
 'https://www.espn.com/soccer/team/stats/_/id/2731/accrington-stanley',
 'https://www.espn.com/soccer/team/stats/_/id/397/barnsley',
 'https://www.espn.com/soccer/team/stats/_/id/642/barrow',
 'https://www.espn.com/soccer/team/stats/_/id/392/birmingham-city',
 'https://www.espn.com/soccer/team/stats/_/id/365/blackburn-rovers',
 'https://www.espn.com/soccer/team/stats/_/id/346/blackpool',
 'https://www.espn.com/soccer/team/stats/_/id/358/bolton-wanderers',
 'https://www.espn.com/soccer/team/stats/_/id/387/bradford-city',
 'https://www.espn.com/soccer/team/stats/_/id/333/bristol-city',
 'https://www.espn.com/soccer/team/stats/_/id/308/bristol-rovers',
 'https://www.espn.com/soccer/team/stats/_/id/2567/burton-albion',
 'https://www.espn.com/soccer/team/stats/_/id/351/cambridge-united',
 'https://www.espn.com/soccer/team/stats/_/id/347/cardiff-city',
 'https://www.espn.com/soccer/team/stats/_/id/322/carlisle-united',
 'http

In [18]:
driver.quit()

In [19]:
def extraer(url):

    # inicia el driver
    driver=webdriver.Chrome(PATH)
    driver.get(url)

    time.sleep(2)

    # acepta cookies
    aceptar=driver.find_element(By.XPATH, '//*[@id="onetrust-accept-btn-handler"]')
    aceptar.click()

    time.sleep(2)
    
    data=[]
    cabeceras=[]
    
    try:
        # dropdown
        dropdown = driver.find_element(By.XPATH, '//*[@id="fittPageContainer"]/div[2]/div[5]/div/div/section/div/div[4]/select[1]')
        select = Select(dropdown)
        select.select_by_visible_text('2022-23')


        time.sleep(1)

        # disciplina
        dis=driver.find_element(By.XPATH, '//*[@id="fittPageContainer"]/div[2]/div[5]/div/div[1]/section/div/div[2]/nav/ul/li[2]/a')
        dis.click()

        time.sleep(2)

        tabla=driver.find_element(By.TAG_NAME, 'tbody')

        filas=tabla.find_elements(By.TAG_NAME, 'tr')

        for f in filas:

            elementos=f.find_elements(By.TAG_NAME, 'td') 

            tmp=[]

            for e in elementos:

                tmp.append(e.text)

            tmp.append(url.split('/')[-1])  # nombre del equipo
            data.append(tmp)


        cabeceras=driver.find_element(By.TAG_NAME, 'thead')

        cabeceras=[c.text for c in cabeceras.find_elements(By.TAG_NAME, 'th')]+['TEAM']

    except:
        time.sleep(0.1)
        
    driver.quit()
    
    return pd.DataFrame(data, columns=cabeceras)

In [20]:
parelelo = Parallel(n_jobs=6, verbose=True)


lst_df = parelelo(delayed(extraer)(url) for url in equipos_stats_urls[:20])

[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
/var/folders/95/ms6dwls51ls1jq0t456d3r200000gn/T/ipykernel_11042/1636708756.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
/var/folders/95/ms6dwls51ls1jq0t456d3r200000gn/T/ipykernel_11042/1636708756.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
/var/folders/95/ms6dwls51ls1jq0t456d3r200000gn/T/ipykernel_11042/1636708756.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
/var/folders/95/ms6dwls51ls1jq0t456d3r200000gn/T/ipykernel_11042/1636708756.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
/var/folders/95/ms6dwls51ls1jq0t456d3r200000gn/T/ipykernel_11042/1636708756.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
/var/folders/95/ms6dwls51ls1jq0t456d3r200000gn/T/ipykernel_11042/16367

In [21]:
len(lst_df)

20

In [22]:
len(equipos_stats_urls)

72

In [23]:
df = pd.concat(lst_df)

In [24]:
df.head()

,RK,NAME,P,YC,RC,PTS,TEAM
0,,,,,,,afc-wimbledon
1,,,,,,,afc-wimbledon
2,,,,,,,afc-wimbledon
3,,,,,,,afc-wimbledon
4,,,,,,,afc-wimbledon


In [25]:
df.shape

(334, 7)

**Tip:** https://pypi.org/project/tqdm/

**Para barras de progreso**

In [26]:
%pip install tqdm

Note: you may need to restart the kernel to use updated packages.


In [27]:
from tqdm.notebook import tqdm    # from tqdm import tqdm   # para .py

In [28]:
lst=[1, 23, 45, 65, 6778, 8756, 32, 45, 67, 89, 99, 9990]

for e in lst:
    print(e**2)

1
529
2025
4225
45941284
76667536
1024
2025
4489
7921
9801
99800100


In [29]:
for e in tqdm(lst):
    print(e**2)

  0%|          | 0/12 [00:00<?, ?it/s]

1
529
2025
4225
45941284
76667536
1024
2025
4489
7921
9801
99800100


In [30]:
parelelo = Parallel(n_jobs=6, verbose=False)


lst_df = parelelo(delayed(extraer)(url) for url in tqdm(equipos_stats_urls[:20]))

  0%|          | 0/20 [00:00<?, ?it/s]

/var/folders/95/ms6dwls51ls1jq0t456d3r200000gn/T/ipykernel_11042/1636708756.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
/var/folders/95/ms6dwls51ls1jq0t456d3r200000gn/T/ipykernel_11042/1636708756.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
/var/folders/95/ms6dwls51ls1jq0t456d3r200000gn/T/ipykernel_11042/1636708756.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
/var/folders/95/ms6dwls51ls1jq0t456d3r200000gn/T/ipykernel_11042/1636708756.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
/var/folders/95/ms6dwls51ls1jq0t456d3r200000gn/T/ipykernel_11042/1636708756.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
/var/folders/95/ms6dwls51ls1jq0t456d3r200000gn/T/ipykernel_11042/1636708756.py:4: DeprecationWarning: executable_path has been deprecated, please